<a href="https://colab.research.google.com/github/Sela80/s/blob/main/Ensemble_de_donn%C3%A9es_de_d%C3%A9tection_de_fraude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#À propos de l'ensemble de données
L'ensemble de données contient les enregistrements des transactions financières à des fins de détection de fraude. (6,3 millions d'enregistrements)
Certains de ces enregistrements ont été signalés comme faux par les algorithmes existants.

D’autres approches pourraient être utilisées pour mettre en avant les propriétés des ingénieurs qui pourraient renforcer davantage les algorithmes de détection de fraude et découvrir où l’algorithme existant présente des lacunes.

ENCAISSEMENT : est le processus d'augmentation du solde du
compte en payant en espèces à un commerçant.

CASH-OUT : est le processus inverse du CASH-IN, il
s'agit de retirer de l'argent chez un commerçant ce qui diminue
le solde du compte.

DÉBIT : il s'agit d'un processus similaire au CASH-OUT et implique l'envoi de l'argent du service d'argent mobile
vers un compte bancaire.

PAIEMENT : est le processus de paiement de biens ou de services aux commerçants qui diminue le solde du compte et augmente le solde du destinataire.

TRANSFERT : est le processus d'envoi d'argent à un autre utilisateur du service via la plateforme d'argent mobile

In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install category_encoders
!pip install category_encoders --upgrade --quiet

In [ ]:
# --- 1. Importation des bibliothèques ---

# 🔧 Manipulation des données
import pandas as pd
import numpy as np

# 📊 Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# ⚙️ Prétraitement et split
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# 🧠 Modèles de classification
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# 🧰 Encodage et transformation
from category_encoders import CatBoostEncoder
from sklearn.compose import ColumnTransformer

# 📈 Évaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

# ⚠️ Divers
import warnings
warnings.filterwarnings('ignore')
import joblib

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/amanalisiddiqui/fraud-detection-dataset")

In [ ]:

# -2. Chargement et Inspection Initiale des Données ---
# Charge le fichier CSV dans un DataFrame Pandas

df = pd.read_csv('/content/fraud-detection-dataset/AIML Dataset.csv')

In [ ]:

# Affiche un titre pour la section
print("=== Informations sur le Dataset ===")
# Affiche les dimensions du dataset (nombre de lignes, nombre de colonnes)
print(f"Dimensions: {df.shape}")

In [ ]:

# Affiche les 5 premières lignes du dataset pour un aperçu
print("\nPremières lignes:")
df.head()

In [ ]:
# Affiche le type de données de chaque colonne (int, float, object, etc.)
print("\nTypes de données:")
print(df.dtypes)

In [ ]:

# Affiche le nombre de valeurs manquantes pour chaque colonne
print("\nValeurs manquantes:")
print(df.isnull().sum())

In [ ]:
#Résumé statistique

df.describe().transpose()

In [ ]:
# Affiche la distribution de la variable cible 'charges'
print("\nDistribution de la variable cible 'isFraud':")
print(df['isFraud'].value_counts())

#-- 2.Visualisons

In [ ]:
df.drop(['nameOrig', 'nameDest', 'step'], axis=1, inplace=True)

In [ ]:
df1=df.copy()

In [ ]:
Num_cols=df1.select_dtypes(include=['int64','float64']).columns
Cat_cols=df1.select_dtypes(include=['object']).columns

In [ ]:
plt.figure(figsize=(11, 8))
# Calcule la matrice de corrélation entre toutes les features
correlation_matrix = df1[Num_cols].corr()
# Affiche la matrice sous forme de heatmap (carte de chaleur)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matrice de Corrélation des Features')
plt.show()

In [ ]:
#Distribution de la variable cible 'isFraud
plt.figure(figsize=(10,6))
sns.histplot(df1['isFraud'], kde=True, bins=50)
plt.title('Distribution de la variable cible isFraud')
plt.xlabel('isFraud ')
plt.show()

# --- 3. Prétraitement des Données ---


In [ ]:
# Selection des variables numériques et catégorielles sans la variable cible(isFraud)

Num_col = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
Num_col.remove('isFraud')

Cat_col = df.select_dtypes(include=['object']).columns.tolist()

print("Numerical Columns (excluding isFraud):", Num_col)
print("Categorical Columns:", Cat_col)

In [ ]:
x=df.drop('isFraud',axis=1)
y=df['isFraud']

In [ ]:
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), Num_col),
    ('cat', CatBoostEncoder(), Cat_col)
])

In [ ]:
# Split train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print("Taille train:", x_train.shape)
print("Taille test:", x_test.shape)

In [ ]:
Régression_logistique = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LogisticRegression())
])

In [ ]:
KNN = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

In [ ]:
DecisionTreeRegressor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

In [ ]:
Random_Forest = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

In [ ]:
XGBClassifier= Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier())
])

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
models_to_evaluate = {
    'Régression_logistique': Régression_logistique,
    'KNN': KNN,
    'DecisionTreeRegressor': DecisionTreeRegressor,
    'Random_Forest': Random_Forest,
    'XGBClassifier': XGBClassifier
}



# --- 5. Évaluation des modèles ---

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

print("\n=== Métriques d'évaluation sur le Test Set ===")

for model_name, model in models_to_evaluate.items():
    print(f"\n--- {model_name} ---")

    # Train the model
    model.fit(x_train, y_train)

    # Make predictions
    y_pred = model.predict(x_test)
    y_pred_proba = model.predict_proba(x_test)[:, 1] # For ROC-AUC

    # Evaluate
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

    # Plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc_score(y_test, y_pred_proba):.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {model_name}')
    plt.legend()
    plt.show()

In [ ]:
joblib.dump(Random_Forest, 'Random_Forest.pkl')
print('modèle Sauvegarde')

In [ ]:
import pickle

# Après avoir entraîné votre modèle Random Forest
with open('Random_Forest.pkl', 'wb') as f:
    pickle.dump(Random_Forest, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from google.colab import files
files.download('Random_Forest.pkl')